#  Proyecto Final

##### Gladys Andrea Rodríguez Guerrero  -  20006107

###### [Video Presentación](https://youtu.be/qs5XbGYX46I)


In [1]:
#conda install -c conda-forge feature_engine
# conda install -c anaconda flask
# conda create -n tf tensorflow
# pip install tensorflow

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer

from sklearn.pipeline import Pipeline

from feature_engine.imputation import(
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder
)

from feature_engine.transformation import LogTransformer

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

import joblib

from keras.models import Model
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasRegressor
from keras.activations import relu, sigmoid
from keras.layers import Dense, Activation
from keras.models import Sequential

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import mean_squared_error

In [2]:
#Librería creada
import my_preprocessors as mypp 

In [3]:
data = pd.read_csv("dataP.csv")
data.head()
data.shape

(246006, 122)

In [4]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']
#cat_vars

#Agregamos variables que por definición son categóricas
cat_vars = cat_vars + ['NAME_CONTRACT_TYPE']+['CODE_GENDER']+['FLAG_OWN_CAR']+['NAME_INCOME_TYPE']+['NAME_EDUCATION_TYPE']+['FLAG_WORK_PHONE']+['FLAG_PHONE']+['REGION_RATING_CLIENT_W_CITY']+['REG_CITY_NOT_LIVE_CITY']+['REG_CITY_NOT_WORK_CITY']+['ORGANIZATION_TYPE']+['FLAG_DOCUMENT_3']

#cat_vars

data[cat_vars] = data[cat_vars].astype('O')
#cat_vars

# Cantidad de variables categóricas
#len(cat_vars)

In [5]:
# Se separa data para train y test
X_train, X_test, y_train, y_test = train_test_split(
                        data.drop(['SK_ID_CURR', 'TARGET'], axis=1),
                        data['TARGET'],
                        test_size=0.15,
                        random_state=2021)

X_train.shape, X_test.shape

((209105, 120), (36901, 120))

In [6]:
# No se usa np.log por datos negatvos, genera error

#y_train = np.sqrt(np.power(y_train,2))
#y_test = np.sqrt(np.power(y_test,2))
#y_train
#y_test

cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]

X_train[cat_vars_with_na].isnull().mean().sort_values(ascending = False)

###### variables tratadas con etiqueta de faltante por cantidad masiva de faltantes.
vars_with_missing_string = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() > 0.2]

#variables tratadas con procedimiento por candiad adecuada de faltantes.
vars_freq_category = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() <= 0.2]

#Faltantes con etiqueta missing
X_train[vars_with_missing_string] = X_train[vars_with_missing_string].fillna('Missing')
X_test[vars_with_missing_string] = X_test[vars_with_missing_string].fillna('Missing')

for var in vars_freq_category:
    mode = X_train[var].mode()[0]
    
    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)

cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]

num_vars = [var for var in X_train.columns
               if var not in cat_vars and var != 'TARGET']

###### Numéricas con faltantes
num_vars_with_na = [var for var in num_vars
                       if X_train[var].isnull().sum() > 0]

for var in num_vars_with_na:
    mean_val = X_train[var].mean()
    
    X_train[var].fillna(mean_val, inplace=True)
    X_test[var].fillna(mean_val, inplace=True)

for var in ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'OWN_CAR_AGE',  'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE',  'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_YEAR']:
    X_train[var] = np.sqrt(np.power(X_train[var],2))
    X_test[var] = np.sqrt(np.power(X_test[var],2))

sesgadas = ['AMT_INCOME_TOTAL',  'DAYS_EMPLOYED', 'YEARS_BEGINEXPLUATATION_AVG', 'NONLIVINGAPARTMENTS_AVG', 'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_MODE', 'YEARS_BEGINEXPLUATATION_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'AMT_REQ_CREDIT_BUREAU_MON']

for var in sesgadas:
    X_train[var] = np.where(X_train[var] == 0, 0, 1)
    X_test[var] = np.where(X_test[var] == 0, 0, 1)

## Configuración del Machine Learning Pipeline

#Variables categoricas a codificar sin ordinalidad
CATEGORICAL_VARS = ['OCCUPATION_TYPE', 'WALLSMATERIAL_MODE', 'FLAG_WORK_PHONE','FLAG_PHONE', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'FLAG_DOCUMENT_3']


#Variables categoricas con NA
#CATEGORICAL_VARS_WITH_NA_FREQUENT = ['OCCUPATION_TYPE'] 

#Variable categoricas con NA pero indicador de Missing
#CATEGORICAL_VARS_WITH_NA_MISSING = ['WALLSMATERIAL_MODE']


#Variables numéricas con NA
NUMERICAL_VARS_WITH_NA = ['EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'DAYS_ID_PUBLISH',
 'DAYS_BIRTH'
]


#Varibles para transformación logaritmia
NUMERICALS_LOG_VARS = ['DAYS_LAST_PHONE_CHANGE']

#Variables para binarización por sesgo fuerte
BINARIZE_VARS = ['OCCUPATION_TYPE',]

#Variables para hacer mapeo categorico por codificación ordinal
NAME_CONTRACT_TYPE_VARS = ['NAME_CONTRACT_TYPE']

CODE_GENDER_VARS = ['CODE_GENDER']

FLAG_OWN_CAR_VARS = ['FLAG_OWN_CAR']

NAME_INCOME_TYPE_VARS = ['NAME_INCOME_TYPE']

NAME_EDUCATION_TYPE_VARS = ['NAME_EDUCATION_TYPE']

#Mapeos de variables categoricas
NAME_CONTRACT_TYPE_MAPPINGS = {'Cash loans':1, 'Revolving loans':0}

CODE_GENDER_MAPPINGS = {'M':1, 'F':0, 'XNA':2}

FLAG_OWN_CAR_MAPPINGS = {'Y':1, 'N':0}

NAME_INCOME_TYPE_MAPPINGS = {'Businessman':0, 'Unemployed':1, 'Commercial associate':2, 'Maternity leave':3, 'Pensioner':4, 'State servant':5, 'Student':6, 'Working':7}

NAME_EDUCATION_TYPE_MAPPINGS = {'Academic degree':0, 'Higher education':1, 'Incomplete higher':2, 'Lower secondary':3, 'Secondary / secondary special':4}

#Variables seleccionadas según análisis de Lasso
FEATURES = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH',
       'FLAG_WORK_PHONE', 'FLAG_PHONE', 'OCCUPATION_TYPE',
       'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'WALLSMATERIAL_MODE',
       'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_3',
       'AMT_REQ_CREDIT_BUREAU_MON']

In [7]:
#Variables categoricas con NA
CATEGORICAL_VARS_WITH_NA_FREQUENT = []

#Variable categoricas con NA pero indicador de Missing
CATEGORICAL_VARS_WITH_NA_MISSING = []


#Variables numéricas con NA
NUMERICAL_VARS_WITH_NA = [
 'DAYS_ID_PUBLISH',
 'DAYS_BIRTH',
'FLAG_DOCUMENT_3',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY'
]


#Variables para binarización por sesgo fuerte
BINARIZE_VARS = [
                ]

#Variables categoricas a codificar sin ordinalidad
CATEGORICAL_VARS = [                   
                    'NAME_CONTRACT_TYPE',
                     'CODE_GENDER',
                     'FLAG_OWN_CAR',
                     'NAME_INCOME_TYPE',
                     'NAME_EDUCATION_TYPE',
                     'ORGANIZATION_TYPE'
]

#Variables seleccionadas según análisis de Lasso
FEATURES =  ['DAYS_ID_PUBLISH',
             'DAYS_BIRTH',
             'NAME_CONTRACT_TYPE',
             'CODE_GENDER',
             'FLAG_OWN_CAR',
             'NAME_INCOME_TYPE',
             'NAME_EDUCATION_TYPE',
             'ORGANIZATION_TYPE',
             'FLAG_DOCUMENT_3',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY'
            ]

In [8]:
#Selección de variables para entrenamiento
X_train = X_train[FEATURES]
X_train

,DAYS_ID_PUBLISH,DAYS_BIRTH,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,ORGANIZATION_TYPE,FLAG_DOCUMENT_3,FLAG_WORK_PHONE,FLAG_PHONE,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY
123235,-1367,-17807,Cash loans,F,N,Working,Secondary / secondary special,Business Entity Type 3,1,1,1,2,0,0
77798,-3560,-11862,Cash loans,M,Y,Working,Secondary / secondary special,Construction,1,1,1,2,0,1
124520,-720,-9616,Cash loans,F,Y,Working,Incomplete higher,Realtor,1,0,0,3,1,1
38155,-1679,-18128,Cash loans,F,Y,Commercial associate,Secondary / secondary special,Other,1,1,0,2,0,0
212930,-3018,-19567,Cash loans,F,N,Working,Secondary / secondary special,Postal,1,1,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133741,-4561,-13744,Cash loans,M,N,Commercial associate,Secondary / secondary special,Business Entity Type 3,0,0,0,2,0,0
214144,-4146,-15264,Cash loans,F,Y,Working,Secondary / secondary special,Hotel,1,1,1,2,0,0
137273,-2100,-10586,Revolving loans,F,Y,Working,Higher education,Industry: type 9,1,0,1,2,0,0
224597,-1007,-8347,Revolving loans,M,N,Working,Secondary / secondary special,Business Entity Type 2,0,0,0,3,0,1


## Machine Learing PipeLine

In [9]:
Target_pipeline = Pipeline([
    
    #============= IMPUTACIONES ===================#
    
    #1. Imputación de varaibles categoricas
    ('missing_imputation', 
         CategoricalImputer(imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)
    ),
    
    #2. Imputación de variables categoricas con NA basado en frequiencia.
    ('frequent_imputation', 
         CategoricalImputer(imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)
    ),
    
    #3. Indicamos Faltante en variables numéricas para imputar
    ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA)),
    
    #4. Imputación de mediana para variables categoricas
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)
    ),
   
    #8. Binarización de Variables con Sesgo Fuerte
    ('binarizer', SklearnTransformerWrapper(
        transformer=Binarizer(threshold=0), variables=BINARIZE_VARS)
    ),
    
    #============ CODIFICACION DE VARIABLES CATEGORICAS NOMINALES ============
    
    ('rare_label_encoder', RareLabelEncoder(
        tol=0.01, n_categories=1, variables=CATEGORICAL_VARS)),
    
    ('categorical_encoder', OrdinalEncoder(
        encoding_method='ordered', variables=CATEGORICAL_VARS)),
    
    #=========== SCALER ==============
    ('scaler', MinMaxScaler()),
    
    #=========== ENTRENAMIENTO DEL MODELO ============
    ('Lasso', Lasso(alpha=0.01, random_state=2022)),
])  

In [10]:
X_train

,DAYS_ID_PUBLISH,DAYS_BIRTH,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,ORGANIZATION_TYPE,FLAG_DOCUMENT_3,FLAG_WORK_PHONE,FLAG_PHONE,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY
123235,-1367,-17807,Cash loans,F,N,Working,Secondary / secondary special,Business Entity Type 3,1,1,1,2,0,0
77798,-3560,-11862,Cash loans,M,Y,Working,Secondary / secondary special,Construction,1,1,1,2,0,1
124520,-720,-9616,Cash loans,F,Y,Working,Incomplete higher,Realtor,1,0,0,3,1,1
38155,-1679,-18128,Cash loans,F,Y,Commercial associate,Secondary / secondary special,Other,1,1,0,2,0,0
212930,-3018,-19567,Cash loans,F,N,Working,Secondary / secondary special,Postal,1,1,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133741,-4561,-13744,Cash loans,M,N,Commercial associate,Secondary / secondary special,Business Entity Type 3,0,0,0,2,0,0
214144,-4146,-15264,Cash loans,F,Y,Working,Secondary / secondary special,Hotel,1,1,1,2,0,0
137273,-2100,-10586,Revolving loans,F,Y,Working,Higher education,Industry: type 9,1,0,1,2,0,0
224597,-1007,-8347,Revolving loans,M,N,Working,Secondary / secondary special,Business Entity Type 2,0,0,0,3,0,1


In [11]:
Target_pipeline.fit(X_train, y_train)

Pipeline(steps=[('missing_imputation', CategoricalImputer(variables=[])),
                ('frequent_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=[])),
                ('missing_indicator',
                 AddMissingIndicator(variables=['DAYS_ID_PUBLISH', 'DAYS_BIRTH',
                                                'FLAG_DOCUMENT_3',
                                                'FLAG_WORK_PHONE', 'FLAG_PHONE',
                                                'REGION_RATING_CLIENT_W_CITY',
                                                'REG_CITY_NOT_LIVE_CITY',
                                                'REG_...
                                  variables=['NAME_CONTRACT_TYPE',
                                             'CODE_GENDER', 'FLAG_OWN_CAR',
                                             'NAME_INCOME_TYPE',
                                             'NAME_EDUCATION_TYPE',
               

In [12]:
# ['FLAG_WORK_PHONE','FLAG_PHONE', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'FLAG_DOCUMENT_3']

#X_train['FLAG_WORK_PHONE'].value_counts()
#X_train['FLAG_WORK_PHONE'].value_counts()
#X_train['FLAG_PHONE'].value_counts()
#X_train['REGION_RATING_CLIENT_W_CITY'].value_counts()
#X_train['REG_CITY_NOT_LIVE_CITY'].value_counts()
#X_train['REG_CITY_NOT_WORK_CITY'].value_counts()
#X_train['ORGANIZATION_TYPE'].value_counts()
#X_train['FLAG_DOCUMENT_3'].value_counts()
#X_train['OCCUPATION_TYPE'].value_counts()


# ['EXT_SOURCE_1',  'EXT_SOURCE_2',  'EXT_SOURCE_3',  'DAYS_LAST_PHONE_CHANGE',  'AMT_REQ_CREDIT_BUREAU_MON',  'DAYS_ID_PUBLISH',  'DAYS_BIRTH' 


#X_train['EXT_SOURCE_1'].value_counts()
#X_train['EXT_SOURCE_2'].value_counts()
#X_train['EXT_SOURCE_3'].value_counts()
#X_train['DAYS_LAST_PHONE_CHANGE'].value_counts()
#X_train['AMT_REQ_CREDIT_BUREAU_MON'].value_counts()
#X_train['DAYS_ID_PUBLISH'].value_counts()
#X_train['DAYS_BIRTH'].value_counts()


#X_train['NAME_CONTRACT_TYPE'].value_counts()
#X_train['CODE_GENDER'].value_counts()
#X_train['FLAG_OWN_CAR'].value_counts()
#X_train['NAME_INCOME_TYPE'].value_counts()
#X_train['NAME_EDUCATION_TYPE'].value_counts()
#X_train['OCCUPATION_TYPE'].dtypes
#X_train['DAYS_BIRTH'].dtypes

In [13]:
#cat_vars_with_na = [var for var in X_train.columns
#                        if X_train[var].isnull().sum() > 0]

#cat_vars_with_na

cat_vars_with_na = [var for var in X_train.columns
                       if X_train[var].dtype == 'O']

cat_vars_with_na

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'ORGANIZATION_TYPE',
 'FLAG_DOCUMENT_3',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY']

In [14]:
#Seleccionamos variables para predicción
X_test = X_test[FEATURES]

In [15]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']
#cat_vars

#Agregamos variables que por definición son categóricas
cat_vars = cat_vars + ['FLAG_MOBIL'] + ['FLAG_EMP_PHONE'] + ['FLAG_WORK_PHONE'] + ['FLAG_CONT_MOBILE'] + ['FLAG_PHONE'] + ['FLAG_EMAIL'] + ['FLAG_WORK_PHONE'] + ['REGION_RATING_CLIENT_W_CITY'] + ['REG_CITY_NOT_LIVE_CITY'] + ['REG_CITY_NOT_WORK_CITY'] + ['ORGANIZATION_TYPE'] + ['FLAG_DOCUMENT_3'] + ['OCCUPATION_TYPE'] + ['WALLSMATERIAL_MODE'] +['NAME_CONTRACT_TYPE'] + ['CODE_GENDER'] + ['FLAG_OWN_CAR'] + ['NAME_INCOME_TYPE'] + ['NAME_EDUCATION_TYPE']
#cat_vars

data[cat_vars] = data[cat_vars].astype('O')
#cat_vars

# Cantidad de variables categóricas
#len(cat_vars)

In [16]:
preds = Target_pipeline.predict(X_test)
preds

array([0.08108367, 0.08108367, 0.08108367, ..., 0.08108367, 0.08108367,
       0.08108367])

In [17]:
#Target_pipeline.transform(X_test)

In [18]:
mean_squared_error(np.exp(y_test), np.exp(preds), squared=False)

0.4717623456916915

In [19]:
X_test

,DAYS_ID_PUBLISH,DAYS_BIRTH,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,ORGANIZATION_TYPE,FLAG_DOCUMENT_3,FLAG_WORK_PHONE,FLAG_PHONE,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY
59464,-2598,-21183,Cash loans,F,N,Pensioner,Secondary / secondary special,XNA,0,0,1,2,0,0
1669,-2179,-12975,Cash loans,F,Y,State servant,Higher education,Government,1,0,1,2,0,0
139512,-4732,-12888,Cash loans,M,Y,State servant,Secondary / secondary special,Police,1,0,0,2,1,1
237760,-359,-7718,Revolving loans,F,Y,Commercial associate,Secondary / secondary special,Business Entity Type 3,0,1,0,1,0,1
11583,-4316,-13459,Cash loans,M,Y,Commercial associate,Higher education,Business Entity Type 3,1,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111417,-4943,-14360,Cash loans,F,Y,Commercial associate,Secondary / secondary special,Trade: type 3,1,0,0,2,0,1
215407,-3623,-20109,Cash loans,F,N,Pensioner,Secondary / secondary special,XNA,0,0,0,2,0,0
145849,-2727,-19231,Cash loans,F,N,Pensioner,Secondary / secondary special,XNA,1,0,1,2,0,0
27637,-3670,-20139,Cash loans,F,N,Pensioner,Secondary / secondary special,XNA,1,0,1,1,0,0


In [20]:
#Guardamos pipeline
joblib.dump(Target_pipeline, 'Target_pipeline.pkl')

['Target_pipeline.pkl']

In [21]:
#X_train